In [2]:
import hashlib

In [1]:
#r,m,v = 7,57,197
#r,m,v = 7,83,283
#r,m,v = 4,19,65
#r, m, v = 7, 10, 30
r, m, v =7, 3, 3
n = m + v
F = GF(2)
L.<a> = GF(2^r)

Clave privada

In [3]:
# Coeficientes de las formas cuadráticas
tt = cputime()
alpha = []
beta = []
#gamma = []
for kk in range(m):
    aux = random_matrix(F,v,n)
    for ii in range(v):
        for jj in range(ii):
            aux[ii,jj] = 0
    alpha += [aux]
    beta += [random_matrix(F,1,n)]
    #gamma += [F.random_element()]
print(cputime(tt))
print alpha
print beta

0.012105
[[0 1 1 0 0 0]
[0 0 0 0 0 1]
[0 0 1 0 1 1], [0 1 0 1 1 1]
[0 1 0 0 1 1]
[0 0 0 0 0 0], [0 1 1 0 1 1]
[0 1 0 1 0 0]
[0 0 1 1 1 0]]
[[0 1 1 0 1 0], [1 1 1 1 0 0], [0 1 0 1 0 1]]


In [4]:
# Matriz de distorsión
tt = cputime()
T = random_matrix(F,v,m)
Tright = block_matrix(F,2,[identity_matrix(F,v),T,zero_matrix(F,m,v),identity_matrix(F,m)])
Tinverse = block_matrix(F,2,[identity_matrix(F,v),-T,zero_matrix(F,m,v),identity_matrix(F,m)])
#Tinverse*Tright == identity_matrix(F,n)
print(cputime(tt))


0.011404
[0 1 0]
[1 0 1]
[1 1 0]


Clave pública

In [6]:
tt = cputime()
Taux = block_matrix(F,2,[identity_matrix(F,v),T.transpose()])
#print Taux == Tright[0:v].transpose()
alpha_pub = [Taux*ele*Tright for ele in alpha]
beta_pub = [ele*Tright for ele in beta]
print((beta_pub[0]).nrows())
gamma_pub = gamma
print(cputime(tt))
print alpha_pub
print alpha_pub

1
0.002333
[[0 1 1 0 1 1]
[0 0 0 0 0 1]
[0 0 1 1 0 1]
[0 0 1 1 0 0]
[0 1 0 1 1 0]
[0 0 0 0 0 1], [0 1 0 0 1 0]
[0 1 0 1 1 0]
[0 0 0 0 0 0]
[0 1 0 1 1 0]
[0 1 0 0 1 0]
[0 1 0 1 1 0], [0 1 1 0 0 0]
[0 1 0 0 0 1]
[0 0 1 0 0 0]
[0 1 1 0 0 1]
[0 1 0 0 0 0]
[0 1 0 0 0 1]]
[[0 1 1 0 1 1]
[0 0 0 0 0 1]
[0 0 1 1 0 1]
[0 0 1 1 0 0]
[0 1 0 1 1 0]
[0 0 0 0 0 1], [0 1 0 0 1 0]
[0 1 0 1 1 0]
[0 0 0 0 0 0]
[0 1 0 1 1 0]
[0 1 0 0 1 0]
[0 1 0 1 1 0], [0 1 1 0 0 0]
[0 1 0 0 0 1]
[0 0 1 0 0 0]
[0 1 1 0 0 1]
[0 1 0 0 0 0]
[0 1 0 0 0 1]]


Algoritmo de firma

In [13]:
mensaje = 'Este mensaje es un mensaje de prueba. Quiero se sea un poco largo para que se aprecie el efecto de la función hash'

In [14]:
#hashed = Integer('0x'+hashlib.sha256(mensaje).hexdigest()).hex()[0:m]
tt = cputime()
hashed = hashlib.sha256(mensaje).hexdigest()[0:m]
message_L = [L(Integer('0x'+hashed[ii]).bits()) for ii in range(len(hashed))]
message_L += [0 for ii in range(m-len(message_L))]
r_v = (L^v).random_element()
#aux = []
#for kk in range(m):
#    aux += [(r_v*(alpha[kk].transpose()[v:].transpose()) + vector(beta[kk].transpose()[v:].transpose())).list()]
mat_coef = matrix([(r_v*(alpha[kk].transpose()[v:].transpose()) + vector(beta[kk].transpose()[v:].transpose())).list() for kk in range(m)])
while mat_coef.is_singular():
    r_v = (L^v).random_element()
    mat_coef = matrix([(r_v*(alpha[kk].transpose()[v:].transpose()) + vector(beta[kk].transpose()[v:].transpose())).list() for kk in range(m)])
r_m = mat_coef.inverse()*(vector(message_L) - matrix([(r_v*(alpha[kk].transpose()[0:v].transpose()) + vector(beta[kk].transpose()[0:v].transpose())).list() for kk in range(m)])*r_v)
signature = Tinverse*vector(r_v.list() + r_m.list())
print(cputime(tt))

0.203184


Algoritmo de verificación

In [23]:
tt = cputime()
verify = True
for kk in range(m):
    print beta_pub[kk]*signature
    verify = verify and (signature*alpha_pub[kk]*signature + (beta_pub[kk]*signature)[0] == message_L[kk])
print(verify)
print(cputime(tt))

(a^4 + a^2)
(a^5 + a^4 + a^2 + 1)
(a^5)
(a^6 + a^5 + a)
(a^6 + a^5 + a^4 + a^3 + a^2 + a + 1)
(a^6 + a^4 + a^2 + a)
(a^6 + a^5 + a^4 + a^3 + a^2 + a + 1)
(a^5 + a^3)
(a^5 + a^4)
(a^5 + 1)
True
0.185095
